In [249]:
import matplotlib.pyplot as plt

import jax.numpy as jnp
from jax.scipy.stats import norm
from jax import grad, jit, vmap
from jax import random
import jax

from typing import Sequence, Callable, Union, Tuple, Optional, Any
from jaxtyping import Array, Float, Int, PyTree

import equinox as eqx
import optax
import chex

from dataclasses import dataclass
from functools import partial

jax.devices()

[StreamExecutorGpuDevice(id=0, process_index=0, slice_index=0)]

In [59]:
def f(x):
    return jnp.sin(x)

fs = vmap(f)

xs = jnp.array([0., jnp.pi / 2, jnp.pi])[..., jnp.newaxis]
ys = fs(xs)

print(xs.shape)
print(ys.shape)

(3, 1)
(3, 1)


In [74]:
primals, f_jvp = jax.jvp(f, (xs,), (jnp.ones_like(xs),))
print(primals)
print(f_jvp)


[[ 0.000000e+00]
 [ 1.000000e+00]
 [-8.742278e-08]]
[[ 1.000000e+00]
 [-4.371139e-08]
 [-1.000000e+00]]


In [41]:
key = random.PRNGKey(0)
key, subkey = random.split(key)

In [42]:
class linear(eqx.Module):
    weight: Array
    bias: Array

    def __init__(self, in_size, out_size, key):
        wkey, bkey = jax.random.split(key)
        self.weight = jax.random.normal(wkey, (out_size, in_size))
        self.bias = jax.random.normal(bkey, (out_size,))

    def __call__(self, x):
        return self.weight @ x + self.bias

In [172]:
def hvp(f, primals, tangents):
    return jax.jvp(jax.grad(f), primals, tangents)

In [278]:

# def f(x):
#   return jnp.asarray(
#     [x[0], 5*x[2], 4*x[1]**2 - 2*x[2], x[2] * jnp.sin(x[0])])
def f(X):
    # return jnp.asarray([jnp.sin(x)])
    return (jnp.sum(jnp.sin(X) ** 2),)



In [267]:
def jacrev_and_vjp(fun: Callable, argnums: Union[int, Sequence[int]] = 0,
           has_aux: bool = False, holomorphic: bool = False, allow_int: bool = False) -> Callable:
  """Jacobian of ``fun`` evaluated row-by-row using reverse-mode AD.

  Args:
    fun: Function whose Jacobian is to be computed.
    argnums: Optional, integer or sequence of integers. Specifies which
      positional argument(s) to differentiate with respect to (default ``0``).
    has_aux: Optional, bool. Indicates whether ``fun`` returns a pair where the
      first element is considered the output of the mathematical function to be
      differentiated and the second element is auxiliary data. Default False.
    holomorphic: Optional, bool. Indicates whether ``fun`` is promised to be
      holomorphic. Default False.
    allow_int: Optional, bool. Whether to allow differentiating with
      respect to integer valued inputs. The gradient of an integer input will
      have a trivial vector-space dtype (float0). Default False.

  Returns:
    A function with the same arguments as ``fun``, that evaluates the Jacobian of
    ``fun`` using reverse-mode automatic differentiation. If ``has_aux`` is True
    then a pair of (jacobian, auxiliary_data) is returned.

  >>> import jax
  >>> import jax.numpy as jnp
  >>>
  >>> def f(x):
  ...   return jnp.asarray(
  ...     [x[0], 5*x[2], 4*x[1]**2 - 2*x[2], x[2] * jnp.sin(x[0])])
  ...
  >>> print(jax.jacrev(f)(jnp.array([1., 2., 3.])))
  [[ 1.       0.       0.     ]
   [ 0.       0.       5.     ]
   [ 0.      16.      -2.     ]
   [ 1.6209   0.       0.84147]]
  """

  docstr = ("Jacobian of {fun} with respect to positional argument(s) "
            "{argnums}. Takes the same arguments as {fun} but returns the "
            "jacobian of the output with respect to the arguments at "
            "positions {argnums}.")

  def jacfun(*args, **kwargs):
    f = jax.scipy.linalg.lu.wrap_init(fun, kwargs)
    f_partial, dyn_args = argnums_partial(f, argnums, args,
                                          require_static_args_hashable=False)
    tree_map(partial(_check_input_dtype_jacrev, holomorphic, allow_int), dyn_args)
    if not has_aux:
      y, pullback = _vjp(f_partial, *dyn_args)
    else:
      y, pullback, aux = _vjp(f_partial, *dyn_args, has_aux=True)
    tree_map(partial(_check_output_dtype_jacrev, holomorphic), y)
    jac = vmap(pullback)(_std_basis(y))
    jac = jac[0] if isinstance(argnums, int) else jac
    example_args = dyn_args[0] if isinstance(argnums, int) else dyn_args
    jac_tree = tree_map(partial(_jacrev_unravel, y), example_args, jac)
    jac_tree = tree_transpose(tree_structure(example_args), tree_structure(y), jac_tree)
    if not has_aux:
      return jac_tree, _vjp
    else:
      return jac_tree, _vjp, aux

  return jacfun

In [326]:
key, subkey = random.split(key, 2)
model = eqx.nn.MLP(key=subkey, in_size=3, out_size=1, width_size=20, depth=3, activation=jax.nn.elu)

# X = random.normal(subkey1, (3,4))
# V = random.normal(subkey2, (3,4))
# V = jnp.ones((3,4))

# res = hvp(f, (X,), (V,))

# X = random.normal(subkey1, (3,))
X = jnp.ones((3,)) * jnp.pi / 2
V = jnp.ones_like(X)

# X = X[..., jnp.newaxis]

primals, vjp_fun = jax.vjp(model, X)
# J, = vmap(vjp_fun)(jnp.eye(len(primals)))


# hesse_vector

a, b = jax.jvp(vjp_fun, (X,), (jnp.ones_like(X),))

# greeks = jax.jacrev(f)(X)

# greeks, = f_vjp(jnp.ones_like(primals))

# primals, greeks = mjp(f, X, jnp.ones_like(primals))


# t_greeks, = vmap(f_vjp)(jnp.eye(3))
# second = jax.jvp(f_vjp, (X,))


print("-" * 80)
print(primals.shape)
print(primals)
print(J)
# print(vmap(f_vjp)(jnp.eye(1)))
print("-" * 80)


print(X.shape)
print(res[0].shape)
print(res[1].shape)
# print(res.shape)
print(res)

ValueError: Shape of cotangent input to vjp pullback function (3,) must be the same as the shape of corresponding primal input (1,).

In [225]:
jax.jvp(jnp.sin, (jnp.pi / 2, ), (1.0, ))

(DeviceArray(1., dtype=float32, weak_type=True),
 DeviceArray(-4.371139e-08, dtype=float32, weak_type=True))

In [161]:
class MakeScalar(eqx.Module):
    model: eqx.Module

    def __call__(self, *args, **kwargs):
        out = self.model(*args, **kwargs)
        return jnp.reshape(out, ())

model = eqx.nn.MLP(key=subkey, in_size=1, out_size=1, width_size=20, depth=3, activation=jax.nn.elu)
print(model(jnp.array([1.0])))


xs_train = jnp.array([[1.0], [2.0], [3.0]])
print(vmap(model)(xs_train))

print("test")
g_model = grad(MakeScalar(model))

print(g_model(jnp.array([1.0])))
print(vmap(g_model)(xs_train))


test = jax.jacrev(vmap(model))(jnp.array([[1.0], [2.0], [3.0]]))
print(test)

X = xs_train
Y = jnp.ones_like(xs_train)
res = hvp(f, (X, ), (Y, ))
print(res)
# vmap(grad(model))(jnp.array([1.0]))

# primals, f_vjp = eqx.filter_vjp(model, jnp.array([1.0]))

# print(primals)
# jax.jacrev(f)(jnp.array([1.0, 2.0, 3.0]))

# res = jax.jacobian(vmap(model))(jnp.array([0.0, jnp.pi/2, jnp.pi, 2*jnp.pi]))
# res = vmap(jax.jacobian(model))(jnp.array([0.0, jnp.pi/2]))

# print(res.shape)
# print(res)


# model = eqx.nn.MLP(key=subkey, in_size=1, out_size=1, width_size=20, depth=3, activation=jax.nn.elu)

# print(model(jnp.array([1.0])).shape)
# x_test = jnp.array([[1.0], [2.0]])

# vmap(model)(x_test)

[0.20363042]
[[0.20363042]
 [0.20618227]
 [0.20519818]]
test
[0.01606509]
[[ 0.01606509]
 [-0.00216613]
 [ 0.00098517]]
[[[[ 0.01606509]
   [ 0.        ]
   [ 0.        ]]]


 [[[ 0.        ]
   [-0.00216612]
   [ 0.        ]]]


 [[[ 0.        ]
   [ 0.        ]
   [ 0.00098517]]]]


TypeError: Argument '<function f at 0x0000020EB29E5550>' of type <class 'function'> is not a valid JAX type.

In [25]:
def loss_fn(model: eqx.nn.MLP, x: Float[Array, "batch"], y: Float[Array, "batch"]) -> Float[Array, ""]:
    pred_y = vmap(model)(x)
    return jnp.mean((y - pred_y) ** 2)

In [26]:
vmap(jax.jacrev(model))(xs)

DeviceArray([[[ 0.00450898]],

             [[-0.00172892]],

             [[ 0.00161424]]], dtype=float32)

In [58]:
jax.hessian(f)(jnp.pi)

DeviceArray(8.742278e-08, dtype=float32, weak_type=True)

In [37]:
jnp.sin(jnp.pi)

DeviceArray(-8.742278e-08, dtype=float32, weak_type=True)